In [3]:
from databricks.connect import DatabricksSession

spark = DatabricksSession.builder.getOrCreate()

In [5]:
old_external_location_name = 'eo000_ext_loc_ctg2'
old_catalog_name = 'eo000_ctg_ext_loc2'

new_storage_credential_name = 'field_demos_credential'
new_external_location_name = 'eo000_ext_loc_ctg5'
new_external_location_url = 'abfss://eo000ext5@oneenvadls.dfs.core.windows.net/'

new_catalog_name = 'eo000_ctg_ext_loc5'

In [13]:
from pyspark.sql.utils import AnalysisException
from databricks.sdk.core import DatabricksError
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import catalog
from typing import *
from termcolor import colored, cprint
import re

class MigrateCatalog():
  def __init__(self, 
               old_external_location_name: str,
               old_catalog_name: str,
               new_storage_credential_name: str,
               new_external_location_name: str,
               new_external_location_url: str,
               new_catalog_name: str,
               db_dict: Optional[Dict[str, List]]) -> None:
    self.w = WorkspaceClient()
    self.old_external_location_name = old_external_location_name
    self.old_catalog_name = old_catalog_name
    self.new_storage_credential_name = new_storage_credential_name
    self.new_external_location_name = new_external_location_name
    self.new_external_location_url = new_external_location_url
    self.new_catalog_name = new_catalog_name
    self.db_dict = self._build_location_for_schemas(db_dict or dict())
    self.securable_dict = {
      catalog.SecurableType.EXTERNAL_LOCATION: [self.w.external_locations, 'External location'],
      catalog.SecurableType.CATALOG: [self.w.catalogs, 'Catalog'],
      catalog.SecurableType.SCHEMA: [self.w.schemas, 'Schema'],
      catalog.SecurableType.TABLE: [self.w.tables, 'Table']
                           }


  def _build_location_for_schemas(self, db_dict: Dict[str, List]) -> Dict[str, List]:
    
    db_dict_out = {}
    if db_dict:
      print('Creating external locations for the schemas based on the input dictionary.')
    for db_name, (ext_loc_name, cred_name, url) in db_dict.items():
      try:
        db_external_location = self.w.external_locations.get(ext_loc_name)
        print(f'\tExternal location {ext_loc_name} already exists and will be used for {db_name}')
      except Exception as e:
        print(f'\tCreating External location {ext_loc_name} ...', end=' ')
        try:
          db_external_location = self.w.external_locations.create(name=ext_loc_name,
                                                        credential_name=cred_name,
                                                        url=url)
          print('DONE!')
        except Exception as e:
          self._print_message(f'\n\t\t{str(e)}')
      finally:
        db_dict_out[db_name] = db_external_location.url
    
    return db_dict_out

  
  def _print_message(self, message: str) -> None:
    cprint(message, 'red', 'on_yellow')


  def _migrate_tags(self, securable_type_str: str,
                  old_catalog_name: str, 
                  new_securable_full_name: str) -> bool:
    try:
      securable_tag_list = spark.sql(f"""
      SELECT * FROM 
        system.information_schema.{securable_type_str.lower()}_tags 
      WHERE catalog_name = '{old_catalog_name}'
              """).collect()
      

      for row in securable_tag_list:
        if securable_type_str.lower() == 'column':
          spark.sql(f"""
                  ALTER TABLE {new_securable_full_name}
                  ALTER COLUMN {row.column_name}
                  SET TAGS ('{row.tag_name}' = '{row.tag_value}')
                  """)
        else:
          spark.sql(f"""
                  ALTER {securable_type_str} {new_securable_full_name}
                  SET TAGS ('{row.tag_name}' = '{row.tag_value}')
                  """)
          
    except Exception as e:
      self._print_message(f'\t\t{str(e)}')


  def _parse_transfer_permissions(self, 
                                  securable_type: catalog.SecurableType, 
                                  old_securable_full_name: str, 
                                  new_securable_full_name: str) -> bool:
    try:
      grants = self.w.grants.get(securable_type=securable_type, 
                                full_name=f'{old_securable_full_name}')
      if grants.privilege_assignments == None:
        return True
      changes = []
      for principal_permission_pair in grants.privilege_assignments:
        principal = principal_permission_pair.principal
        privileges = [eval('catalog.Privilege.'+privilege) for privilege in principal_permission_pair.privileges if (('VOL' not in privilege) and ('BROWSE' not in privilege))]
        changes.append(catalog.PermissionsChange(add=privileges, principal=principal))
      self.w.grants.update(full_name=new_securable_full_name, 
                          securable_type=securable_type, 
                          changes=changes)
      return True
    except Exception as e:
      self._print_message(str(e))
      return False
  

  
  def _get_or_create_transfer(self, 
                      securable_type: catalog.SecurableType, 
                      old_securable_full_name: str,
                      new_securable_full_name: str,
                      **kwarg) -> None:
    new_securable = None
    analysis_exception_hit = 0
    new_securable_name = re.findall('[^.]+$', new_securable_full_name)[0]
    try:
      new_securable = self.securable_dict[securable_type][0].get(new_securable_full_name)
      print(f'{self.securable_dict[securable_type][1]} {new_securable_name} already exists. Only transferring permissions, comments and tags ...', end=' ')
    except Exception as e:
      print(f'Creating {self.securable_dict[securable_type][1]} {new_securable_name} and transferring permissions, comments and tags ...', end=' ')
      try:
        if securable_type == catalog.SecurableType.TABLE:
          tbl = self.securable_dict[securable_type][0].get(old_securable_full_name)
          spark.sql(f'CREATE TABLE {new_securable_full_name} DEEP CLONE {tbl.full_name}')
          new_securable = self.securable_dict[securable_type][0].get(full_name=new_securable_full_name)
          spark.sql(f'COMMENT ON TABLE {new_securable_full_name} IS "{tbl.comment}"')
        else:
          new_securable = self.securable_dict[securable_type][0].create(
            name=new_securable_name,
            comment=self.securable_dict[securable_type][0].get(old_securable_full_name).comment,
            **kwarg)
      except AnalysisException as ae:
        analysis_exception_hit = 1
        self._print_message(f'\n\t\t{str(ae)}')
    
    finally:
      if not analysis_exception_hit:
        _ = self._parse_transfer_permissions(securable_type=securable_type, 
                              old_securable_full_name=old_securable_full_name,
                              new_securable_full_name=new_securable_full_name)  
        if securable_type != catalog.SecurableType.EXTERNAL_LOCATION:     
          _ = self._migrate_tags(self.securable_dict[securable_type][1],
                  self.old_catalog_name, 
                  new_securable_full_name)
        if securable_type == catalog.SecurableType.TABLE:
          for col in tbl.columns:
            spark.sql(f"""
                    ALTER TABLE {new_securable_full_name}
                    ALTER COLUMN {col.name}
                    COMMENT "{col.comment}"
                    """)
          _ = self._migrate_tags('column',
                  self.old_catalog_name, 
                  new_securable_full_name)
        print('DONE!')  
    
    return new_securable
  
  
  
  def __call__(self):
    self.new_external_location = self._get_or_create_transfer(
                        catalog.SecurableType.EXTERNAL_LOCATION,
                        self.old_external_location_name,
                        self.new_external_location_name,
                        credential_name=self.new_storage_credential_name,
                        url=self.new_external_location_url)    

    print('\t', end='')
    self.new_catalog = self._get_or_create_transfer(
                        catalog.SecurableType.CATALOG,
                        self.old_catalog_name,
                        self.new_catalog_name,
                        storage_root=self.new_external_location.url) 
         
    
    db_list = self.w.schemas.list(self.old_catalog_name)
    for db in db_list: 
      print('\t', end='\t')         
      self.new_db = self._get_or_create_transfer(
                        catalog.SecurableType.SCHEMA,
                        f'{self.old_catalog_name}.{db.name}',
                        f'{self.new_catalog_name}.{db.name}',
                        catalog_name=self.new_catalog.name,
                        storage_root=self.db_dict.get(db.name, None))
      
      tbl_list = self.w.tables.list(catalog_name=self.old_catalog_name, schema_name=db.name)
      for tbl in tbl_list:
        if tbl.table_type == catalog.TableType.MANAGED:
          print('\t', end='\t')
          self.new_table = self._get_or_create_transfer( 
                      catalog.SecurableType.TABLE, 
                      f'{self.old_catalog_name}.{db.name}.{tbl.name}',
                      f'{self.new_catalog_name}.{db.name}.{tbl.name}')
          
          
          

In [18]:
inputs = dict(
    old_external_location_name = 'eo000_ext_loc_ctg2',
    old_catalog_name = 'eo000_ctg_ext_loc2',
    new_storage_credential_name = 'field_demos_credential',
    new_external_location_name = 'eo000_ext_loc_ctg5',
    new_external_location_url = 'abfss://eo000ext5@oneenvadls.dfs.core.windows.net/',
    new_catalog_name = 'eo000_ctg_ext_loc5')

db_dict = {
    'db1': ['eo000_ext_db_loc01', 'field_demos_credential', 'abfss://eo000extdb0@oneenvadls.dfs.core.windows.net/'],
    'db2': ['eo000_ext_db_loc01', 'field_demos_credential', 'abfss://eo000extdb0@oneenvadls.dfs.core.windows.net/'],
    'db_empty': ['eo000_ext_db_loc02', 'field_demos_credential', 'abfss://eo000extdb1@oneenvadls.dfs.core.windows.net/']
}

In [19]:
migrate = MigrateCatalog(**inputs, db_dict=db_dict)
migrate()

Creating external locations for the schemas based on the input dictionary.
	External location eo000_ext_db_loc01 already exists and will be used for db1
	External location eo000_ext_db_loc01 already exists and will be used for db2
	External location eo000_ext_db_loc02 already exists and will be used for db_empty
External location eo000_ext_loc_ctg5 already exists. Only transferring permissions, comments and tags ... DONE!
	Creating Catalog eo000_ctg_ext_loc5 and transferring permissions, comments and tags ... DONE!
		Creating Schema db1 and transferring permissions, comments and tags ... DONE!
		Creating Table tbl1 and transferring permissions, comments and tags ... DONE!
		Creating Table tbl1_new and transferring permissions, comments and tags ... DONE!
		Creating Schema db2 and transferring permissions, comments and tags ... DONE!
		Creating Schema db_empty and transferring permissions, comments and tags ... DONE!
		Creating Schema db_ext and transferring permissions, comments and ta